In [1]:
%load_ext autoreload
%autoreload 2

# 1. Patch descriptors register in 3D for 642 templates

In [2]:
import glob
import os

# Load templates
template_path = "datasets/bop23_challenge/datasets/templates_pyrender/icbin_642/obj_000001"
template_files = sorted(glob.glob(os.path.join(template_path, "*.png")), key=os.path.getmtime)

In [3]:
from PIL import Image
import numpy as np

templates = [np.array(Image.open(template_file).convert("RGB").resize((420,420)))[:,:,:3] for template_file in template_files] # This image has 4 channels- the last one is not crucial - maybe about opacity

In [4]:
import torchvision.transforms as T
import torch.nn.functional as F
import torch

def patches_feature_extraction(template_patches, dinov2_vitl14, device):
    # crop_rgb: numpy array
    # temps = np.transpose(np.array(template_patches), (0,2,3,1))
    rgb_normalize = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    )
    normalized_patches = torch.stack([rgb_normalize(patch) for patch in template_patches])
    layers_list = list(range(18))
    torch.cuda.empty_cache()
    with torch.no_grad(): 
        feature_patches= dinov2_vitl14.module.get_intermediate_layers(normalized_patches.to(device), n=layers_list, reshape=True)
    return feature_patches

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
dinov2_vitl14.patch_size = 14
if torch.cuda.is_available():
    dinov2_vitl14 = torch.nn.DataParallel(dinov2_vitl14).to(device)  # Use DataParallel for multiple GPUs

Using cache found in /home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main
/home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


In [6]:
batch_size = 100
template_batches = [templates[i:i+batch_size] for i in range(0, len(templates), batch_size)]
patch_features= list()

for batch in template_batches:
    torch.cuda.empty_cache()
    batch_feature = patches_feature_extraction(batch, dinov2_vitl14, device)
    patch_features.append(batch_feature[0].to('cpu'))
    del batch_feature
    
patch_features = torch.cat(patch_features).permute(0,2,3,1).view(-1,30*30,1024)

In [7]:
del dinov2_vitl14

In [8]:
patch_features.shape

torch.Size([642, 900, 1024])

In [9]:
# Given only 30 first patches are valid
valid_patch_features = patch_features[:, :30,].reshape(-1,1024)
valid_patch_features.shape

torch.Size([19260, 1024])

In [20]:
# Number of valid patches on each template - here given 30 for each templates
num_valid_patches = [30]*642
print(len(num_valid_patches), num_valid_patches)

642 [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,

In [10]:
import numpy as np
from sklearn.decomposition import PCA

# Apply PCA with the 256 components to reduce dimensionality of the features
pca = PCA(n_components=256)
pca_patches_descriptors = pca.fit_transform(np.array(valid_patch_features.cpu()))
pca_patches_descriptors.shape

(19260, 256)

# 2. Perform Kmean clustering for all patch descriptors from templates (2048 clusters)

In [19]:
# https://github.com/facebookresearch/faiss/wiki/Faiss-building-blocks:-clustering,-PCA,-quantization
import faiss
ncentroids = 2048
niter = 20
verbose = True
d = pca_patches_descriptors.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose, gpu=True)
kmeans.train(pca_patches_descriptors)

Clustering 19260 points in 256D to 2048 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
  Iteration 19 (0.13 s, search 0.08 s): objective=4683.79 imbalance=1.247 nsplit=0       


WARNING clustering 19260 points to 2048 centroids: please provide at least 79872 training points


4683.7861328125

In [25]:
# Assign labels to the data points
labels = kmeans.index.search(pca_patches_descriptors, 1)[1]
labels.shape

(19260, 1)

In [33]:
templates_labels = list()
start_idx = 0
for num in num_valid_patches:
    end_idx = start_idx + num
    template_labels = labels[start_idx:end_idx].reshape(-1)
    templates_labels.append(template_labels)
    start_idx = end_idx

In [43]:
import math

def calculate_templates_vector(templates_labels, num_clusters = 2048):
    # Calculate bag-of-words descriptors of the templates

    templates_vector = list()
    all_occurrences = [np.bincount(templates_label, minlength=2048) for templates_label in templates_labels]
    ni_array = np.sum(np.array(all_occurrences), axis = 0)
    N = len(templates_labels) # Number of templates
    for t in range(len(templates_labels)):
        template_vector = list()
        occurrences = np.bincount(templates_labels[t], minlength=2048)
        for i in range(num_clusters):
            n_it = occurrences[i]
            nt = len(templates_labels[t])
            ni = ni_array[i]
            bi = n_it / nt * math.log(N / ni)
            template_vector.append(bi)
        templates_vector.append(np.array(template_vector))
    return templates_vector
templates_vector = calculate_templates_vector(templates_labels = templates_labels, num_clusters = 2048)

In [53]:
np.max(templates_vector[0])

0.19238137077100054

# 3. Retrieving similar templates

In [54]:
# Load image crop 
crop_rgb = np.array(Image.open("cnos_analysis/crop_proposals/crop1.png").convert("RGB").resize((420,420))) # (124, 157, 3)

In [57]:
crop_rgb.shape

(420, 420, 3)

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
dinov2_vitl14.patch_size = 14
if torch.cuda.is_available():
    dinov2_vitl14 = torch.nn.DataParallel(dinov2_vitl14).to(device)  # Use DataParallel for multiple GPUs

Using cache found in /home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main


In [73]:
crop_feature = patches_feature_extraction(np.expand_dims(crop_rgb, 0), dinov2_vitl14, device)

In [74]:
del dinov2_vitl14

In [85]:
valid_crop_features = crop_feature[0].permute(0,2,3,1).reshape(30*30,-1)[:400,:]
valid_crop_features.shape

torch.Size([400, 1024])

In [86]:
from sklearn.decomposition import PCA

pca = PCA(n_components=256)
pca_crop_patches_descriptors = pca.fit_transform(np.array(valid_crop_features.cpu()))
pca_crop_patches_descriptors.shape

(400, 256)

In [93]:
# Assign labels to the data points
crop_labels = kmeans.index.search(pca_crop_patches_descriptors, 1)[1].reshape(-1)
crop_labels.shape

(400,)

In [94]:
crop_vector = calculate_templates_vector(templates_labels = [crop_labels], num_clusters = 2048)


/tmp/ipykernel_350848/2790732940.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  bi = n_it / nt * math.log(N / ni)
/tmp/ipykernel_350848/2790732940.py:17: RuntimeWarning: invalid value encountered in scalar multiply
  bi = n_it / nt * math.log(N / ni)


In [97]:
import math
# For word_i, term frequency = occurences of word_i within the crop / number of occurences of word_i in all templates). 
def calculate_crop_vector(crop_labels, templates_labels, num_clusters = 2048):
    # Calculate bag-of-words descriptors of the templates
    all_occurrences_crop = np.bincount(crop_labels, minlength=2048)

    all_occurrences_templates = [np.bincount(templates_label, minlength=2048) for templates_label in templates_labels]
    ni_array = np.sum(np.array(all_occurrences_templates), axis = 0)
    N = len(templates_labels) # Number of templates = 642 

    crop_vector = list()
    for i in range(num_clusters):
        n_it = all_occurrences_crop[i]
        nt = crop_labels.shape[0] # Number of words in crop = 400 
        ni = ni_array[i]
        bi = n_it / nt * math.log(N / ni)
        crop_vector.append(bi)
    return crop_vector
crop_vector = calculate_crop_vector(crop_labels = crop_labels, templates_labels = templates_labels, num_clusters = 2048)

In [108]:
# for i, value in enumerate(crop_vector):
#     if value>0:
#         print(i)